In [31]:
#Lauren Todd-Mariano
#Homework 4 - Pandas
#Academy of Py

#import pandas
import pandas as pd

#import numpy, ltm to go back later to see if alternate solution
import numpy as np

#bring in the schools complete file
schools = pd.read_csv("04-Pandas_Homework_PyCitySchools_Resources_schools_complete.csv")

In [32]:
#check the schools complete data
schools.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [33]:
#bring in the students complete file
students = pd.read_csv("04-Pandas_Homework_PyCitySchools_Resources_students_complete.csv")

In [34]:
#check the students completed data
students.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [87]:
#try something different - bring in the student data to schools to address budget issues
combined_schools = schools.merge(students, on='school_name', how='outer')
combined_schools.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [112]:
#make specific required calculations for the table
#add columns

#pass_math
combined_schools['pass_math']=np.where(combined_schools['math_score']>=70,1,0)

#pass_reading
combined_schools['pass_reading']=np.where(combined_schools['reading_score']>=70,1,0)

#number of schools
School_Count=len(combined_schools["School ID"].unique())

#number of students
Student_Count=len(combined_schools["Student ID"].unique())

#average math
Avg_Math=combined_schools["math_score"].mean()

#average reading
Avg_Reading=combined_schools["reading_score"].mean()

#percent pass math
Math_Pass_Rate=combined_schools["pass_math"].mean()*100

#percent pass reading
Reading_Pass_Rate=combined_schools["pass_reading"].mean()*100

#overall pass rate
Overall_Pass_Rate=(Math_Pass_Rate + Reading_Pass_Rate)/2

#District Summary
District_Summary_Table = pd.DataFrame({ "Number of Schools":[School_Count],
                                       "Number of Students":[Student_Count],
                                       "Avg Math Score": [Avg_Math],
                                       "Avg Reading Score": [Avg_Reading],
                                       "% Pass Math": [Math_Pass_Rate],
                                       "% Pass Reading": [Reading_Pass_Rate],
                                       "Overall Pass Rate": [Overall_Pass_Rate]
                                      })
District_Summary_Table

#LTM still working on how to calculate the total budget given the duplicate rows sans pivoting, perhaps groupby

,Number of Schools,Number of Students,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Reading,Overall Pass Rate
0,15,39170,78.985371,81.87784,74.980853,85.805463,80.393158


In [92]:
#one more try using slide 28 pandas day 3
District_Summary_Table_Group=District_Summary_Table.groupby(['school_name'])
District_Summary_Table_Group

In [95]:
print(District_Summary_Table_Group)

In [96]:
District_Summary_Table_Group.head()

,Number of Schools,Number of Students,Avg Math Score,Avg Reading Score,% Pass Math,% Pass Reading,Overall Pass Rate
0,15,39170,78.985371,81.87784,74.980853,85.805463,80.393158


In [117]:
#moving onto school summary
school_summary_table = pd.pivot_table(combined_schools,index=["type","school_name"],
                                      values=["Student ID","budget","math_score","reading_score","pass_math","pass_reading"], 
                                      aggfunc={"Student ID":len,"budget":np.mean,"math_score":np.mean,"reading_score":np.mean,"pass_math":np.mean,"pass_reading":np.mean})
school_summary_table['Per Student Budget']=school_summary_table["budget"]/school_summary_table["Student ID"]
school_summary_table['Combined Pass Rate']=(school_summary_table["pass_math"]+school_summary_table["pass_reading"])/2*100
print(school_summary_table)

                                Student ID   budget  math_score  pass_math  \
type     school_name                                                         
Charter  Cabrera High School          1858  1081356   83.061895   0.941335   
         Griffin High School          1468   917500   83.351499   0.933924   
         Holden High School            427   248087   83.803279   0.925059   
         Pena High School              962   585858   83.839917   0.945946   
         Shelton High School          1761  1056600   83.359455   0.938671   
         Thomas High School           1635  1043130   83.418349   0.932722   
         Wilson High School           2283  1319574   83.274201   0.938677   
         Wright High School           1800  1049400   83.682222   0.933333   
District Bailey High School           4976  3124928   77.048432   0.666801   
         Figueroa High School         2949  1884411   76.711767   0.659885   
         Ford High School             2739  1763916   77.102592 

In [118]:
#top 5 schools based on pass rate
top_5_pass_rate=school_summary_table.sort_values("Combined Pass Rate",ascending=False)
top_5_pass_rate.head()

Student ID   budget  math_score  pass_math  \
type    school_name                                                       
Charter Cabrera High School        1858  1081356   83.061895   0.941335   
        Thomas High School         1635  1043130   83.418349   0.932722   
        Pena High School            962   585858   83.839917   0.945946   
        Griffin High School        1468   917500   83.351499   0.933924   
        Wilson High School         2283  1319574   83.274201   0.938677   

                             pass_reading  reading_score  Per Student Budget  \
type    school_name                                                            
Charter Cabrera High School      0.970398      83.975780               582.0   
        Thomas High School       0.973089      83.848930               638.0   
        Pena High School         0.959459      84.044699               609.0   
        Griffin High School      0.971390      83.816757               625.0   
        Wilson High School       0.965396      83.989488               578.0   

                             Combined Pass Rate  
type    school_name                              
Charter Cabrera High School           95.586652  
        Thomas High School            95.290520  
        Pena High School              95.270270  
        Griffin High School           95.265668  
        Wilson High School            95.203679

In [119]:
#bottom 5 schools based on pass rate
bottom_5_pass_rate=school_summary_table.sort_values("Combined Pass Rate")
bottom_5_pass_rate.head()

Student ID   budget  math_score  pass_math  \
type     school_name                                                         
District Rodriguez High School        3999  2547363   76.842711   0.663666   
         Figueroa High School         2949  1884411   76.711767   0.659885   
         Huang High School            2917  1910635   76.629414   0.656839   
         Johnson High School          4761  3094650   77.072464   0.660576   
         Ford High School             2739  1763916   77.102592   0.683096   

                                pass_reading  reading_score  \
type     school_name                                          
District Rodriguez High School      0.802201      80.744686   
         Figueroa High School       0.807392      81.158020   
         Huang High School          0.813164      81.182722   
         Johnson High School        0.812224      80.966394   
         Ford High School           0.792990      80.746258   

                                Per Student Budget  Combined Pass Rate  
type     school_name                                                    
District Rodriguez High School               637.0           73.293323  
         Figueroa High School                639.0           73.363852  
         Huang High School                   655.0           73.500171  
         Johnson High School                 650.0           73.639992  
         Ford High School                    644.0           73.804308

In [121]:
#math by grade
math_by_grade = pd.pivot_table(combined_schools,index=["school_name","grade"],
                                      values=["math_score","pass_math"], 
                                      aggfunc={"math_score":np.mean,"pass_math":np.mean})
print(math_by_grade)

                             math_score  pass_math
school_name           grade                       
Bailey High School    10th    76.996772   0.663438
                      11th    77.515588   0.684253
                      12th    76.492218   0.642996
                      9th     77.083676   0.671468
Cabrera High School   10th    83.154506   0.939914
                      11th    82.765560   0.923237
                      12th    83.277487   0.950262
                      9th     83.094697   0.952652
Figueroa High School  10th    76.539974   0.665793
                      11th    76.884344   0.653032
                      12th    77.151369   0.685990
                      9th     76.403037   0.641355
Ford High School      10th    77.672316   0.693503
                      11th    76.918058   0.687405
                      12th    76.179963   0.654917
                      9th     77.361345   0.689076
Griffin High School   10th    84.229064   0.940887
                      11th    8

In [123]:
#create spending per student buckets for the school_summary_table
bins = [575,600,625,650,675]
group_names = ["lowest","low-mid","high-mid","highest"]
school_summary_table['budget_range']=pd.cut(school_summary_table['Per Student Budget'],bins,labels=group_names)

In [124]:
print(school_summary_table)

                                Student ID   budget  math_score  pass_math  \
type     school_name                                                         
Charter  Cabrera High School          1858  1081356   83.061895   0.941335   
         Griffin High School          1468   917500   83.351499   0.933924   
         Holden High School            427   248087   83.803279   0.925059   
         Pena High School              962   585858   83.839917   0.945946   
         Shelton High School          1761  1056600   83.359455   0.938671   
         Thomas High School           1635  1043130   83.418349   0.932722   
         Wilson High School           2283  1319574   83.274201   0.938677   
         Wright High School           1800  1049400   83.682222   0.933333   
District Bailey High School           4976  3124928   77.048432   0.666801   
         Figueroa High School         2949  1884411   76.711767   0.659885   
         Ford High School             2739  1763916   77.102592 

In [134]:
print("It is interesting that the charter schools are out-perfomring the district schools.")

It is interesting that the charter schools are out-perfomring the district schools.


In [129]:
#scores by budget
scores_by_budget=pd.pivot_table(school_summary_table,index=["budget_range"],
                                      values=["math_score","pass_math","reading_score","pass_reading"], 
                                      aggfunc={"math_score":np.mean,"pass_math":np.mean,"reading_score":np.mean,"pass_reading":np.mean})
print(scores_by_budget)

              math_score  pass_math  pass_reading  reading_score
budget_range                                                    
lowest         83.436210   0.935415      0.964596      83.892196
low-mid        83.595708   0.939935      0.965425      83.930728
high-mid       78.032719   0.711124      0.834538      81.416375
highest        76.959583   0.662184      0.810897      81.058567


In [136]:
print("It is even more interesting that the low budget range schools are out-performing the higher budget range schools.")

It is even more interesting that the low budget range schools are out-performing the higher budget range schools.


In [130]:
#create 3 school size buckets for school_summary_table
bins = [0,1750,3500,5250]
group_names = ["smallest","mid-sized","largest"]
school_summary_table['school_size']=pd.cut(school_summary_table['Student ID'],bins,labels=group_names)

In [131]:
print(school_summary_table)

                                Student ID   budget  math_score  pass_math  \
type     school_name                                                         
Charter  Cabrera High School          1858  1081356   83.061895   0.941335   
         Griffin High School          1468   917500   83.351499   0.933924   
         Holden High School            427   248087   83.803279   0.925059   
         Pena High School              962   585858   83.839917   0.945946   
         Shelton High School          1761  1056600   83.359455   0.938671   
         Thomas High School           1635  1043130   83.418349   0.932722   
         Wilson High School           2283  1319574   83.274201   0.938677   
         Wright High School           1800  1049400   83.682222   0.933333   
District Bailey High School           4976  3124928   77.048432   0.666801   
         Figueroa High School         2949  1884411   76.711767   0.659885   
         Ford High School             2739  1763916   77.102592 

In [132]:
#scores by school size
scores_by_school_size=pd.pivot_table(school_summary_table,index=["school_size"],
                                      values=["math_score","pass_math","reading_score","pass_reading"], 
                                      aggfunc={"math_score":np.mean,"pass_math":np.mean,"reading_score":np.mean,"pass_reading":np.mean})
print(scores_by_school_size)

             math_score  pass_math  pass_reading  reading_score
school_size                                                    
mid-sized     83.603261   0.934412      0.966617      83.881343
smallest      80.545935   0.821691      0.896286      82.676142
largest       77.063340   0.664643      0.810597      80.919864


In [133]:
#scores by school type
scores_by_school_type=pd.pivot_table(school_summary_table,index=["type"],
                                      values=["math_score","pass_math","reading_score","pass_reading"], 
                                      aggfunc={"math_score":np.mean,"pass_math":np.mean,"reading_score":np.mean,"pass_reading":np.mean})
print(scores_by_school_type)

          math_score  pass_math  pass_reading  reading_score
type                                                        
Charter    83.473852   0.936208      0.965865      83.896421
District   76.956733   0.665485      0.807991      80.966636
